# Fetch Rewards Coding Exercise - Data Analyst

In [24]:
# importing all the required packages
import os
import json
import gzip

from urllib.request import urlopen

# dataframe and series 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from datetime import *

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib

import warnings
warnings.filterwarnings('ignore')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

plt.style.use('fivethirtyeight')
%matplotlib inline

In [25]:
#%%
#Setting Warnings Settings 
warnings.filterwarnings(action='once') 
pd.set_option('display.float_format', lambda x: '%.3f' % x) #display changed from scientific to numeric format
pd.set_option('expand_frame_repr', False)

In [42]:
filelist = {"usersdf":"users.json.gz","receiptsdf":"receipts.json.gz","brandsdf":"brands.json.gz"}

## Users data:  Conversion and formatting

In [44]:
df_data = []
with gzip.open("users.json.gz") as data:
    for i in data:
        df_data.append(json.loads(i.strip()))
    usersdf = pd.json_normalize(df_data)

In [49]:
# Renaming the columns with correct format
usersdf = usersdf.rename(columns={'_id.$oid':'user_id',
                                  'createdDate.$date':'createdDate',
                                  'lastLogin.$date':'lastLoginDate'}
                        )
usersdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   active         495 non-null    bool   
 1   role           495 non-null    object 
 2   signUpSource   447 non-null    object 
 3   state          439 non-null    object 
 4   user_id        495 non-null    object 
 5   createdDate    495 non-null    int64  
 6   lastLoginDate  433 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [55]:
# converting dates to correct datetime format
usersdf['createdDate']=pd.to_datetime(usersdf['createdDate'], unit='ns')
usersdf['lastLoginDate']=pd.to_datetime(usersdf['lastLoginDate'], unit='ns')
usersdf.head(2)

,active,role,signUpSource,state,user_id,createdDate,lastLoginDate
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1970-01-01 00:26:49.687444800,1970-01-01 00:26:49.687537858
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1970-01-01 00:26:49.687444800,1970-01-01 00:26:49.687537858


## Brands data:  Conversion and formatting

In [62]:
df_data = []
with gzip.open("brands.json.gz") as data:
    for i in data:
        df_data.append(json.loads(i.strip()))
    brandsdf = pd.json_normalize(df_data)

In [63]:
brandsdf=brandsdf.rename(columns={'_id.$oid':'brand_id',brandsdf.columns[6]: 'cpg_id','cpg.$ref' : 'cpg_ref'})
brandsdf.head(2)

,barcode,category,categoryCode,name,topBrand,brand_id,cpg_id,cpg_ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS


## Receipts data:  Conversion and formatting

In [93]:
df_data = []
with gzip.open("receipts.json.gz") as data:
    for i in data:
        df_data.append(json.loads(i.strip()))
    receiptsdf = pd.json_normalize(df_data)
receiptsdf.head(2)

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.000,"Receipt number 2 completed, bonus point schedu...",500.0,5.000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1609687531000.000,1609687536000,1609687531000.000,1609632000000.000
1,150.000,"Receipt number 5 completed, bonus point schedu...",150.0,2.000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1609687483000.000,1609687488000,1609687483000.000,1609601083000.000


In [94]:
receiptsdf=receiptsdf.rename(columns= {
        '_id.$oid' : 'receiptId',
        'createDate.$date' : 'receipt_createDate',
        'dateScanned.$date' : 'receipt_ScannedDate',
        'finishedDate.$date' : 'receipt_finishedDate',
        'modifyDate.$date' : 'receipt_modifyDate',
        'pointsAwardedDate.$date' : 'pointsAwardedDate',
        'purchaseDate.$date' : 'purchaseDate'
    
})

In [95]:
# converting dates to the correct datetime format
receiptsdf['receipt_createDate']=pd.to_datetime(receiptsdf['receipt_createDate'], unit='ns')
receiptsdf['receipt_ScannedDate']=pd.to_datetime(receiptsdf['receipt_ScannedDate'], unit='ns')
receiptsdf['receipt_finishedDate']=pd.to_datetime(receiptsdf['receipt_finishedDate'], unit='ns')
receiptsdf['receipt_modifyDate']=pd.to_datetime(receiptsdf['receipt_modifyDate'], unit='ns')
receiptsdf['pointsAwardedDate']=pd.to_datetime(receiptsdf['pointsAwardedDate'], unit='ns')
receiptsdf['purchaseDate']=pd.to_datetime(receiptsdf['purchaseDate'], unit='ns')
receiptsdf.head(2)

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,receiptId,receipt_createDate,receipt_ScannedDate,receipt_finishedDate,receipt_modifyDate,pointsAwardedDate,purchaseDate
0,500.000,"Receipt number 2 completed, bonus point schedu...",500.0,5.000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1970-01-01 00:26:49.687531,1970-01-01 00:26:49.687531,1970-01-01 00:26:49.687531,1970-01-01 00:26:49.687536,1970-01-01 00:26:49.687531,1970-01-01 00:26:49.632000
1,150.000,"Receipt number 5 completed, bonus point schedu...",150.0,2.000,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1970-01-01 00:26:49.687483,1970-01-01 00:26:49.687483,1970-01-01 00:26:49.687483,1970-01-01 00:26:49.687488,1970-01-01 00:26:49.687483,1970-01-01 00:26:49.601083


## Conversion of nested JSON inside the Receipt data into a new df
- There can be 2 approaches to flatten "rewardsReceiptItemList" nested JSON
    - Converting current receiptdf to more granular version with multiple order quantities
    - Create new Dataframe which will have all the order and quantity details

In [96]:
receiptsdf.rewardsReceiptItemList[1]

[{'barcode': '4011',
  'description': 'ITEM NOT FOUND',
  'finalPrice': '1',
  'itemPrice': '1',
  'partnerItemId': '1',
  'quantityPurchased': 1},
 {'barcode': '028400642255',
  'description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ',
  'finalPrice': '10.00',
  'itemPrice': '10.00',
  'needsFetchReview': True,
  'needsFetchReviewReason': 'USER_FLAGGED',
  'partnerItemId': '2',
  'pointsNotAwardedReason': 'Action not allowed for user and CPG',
  'pointsPayerId': '5332f5fbe4b03c9a25efd0ba',
  'preventTargetGapPoints': True,
  'quantityPurchased': 1,
  'rewardsGroup': 'DORITOS SPICY SWEET CHILI SINGLE SERVE',
  'rewardsProductPartnerId': '5332f5fbe4b03c9a25efd0ba',
  'userFlaggedBarcode': '028400642255',
  'userFlaggedDescription': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ',
  'userFlaggedNewItem': True,
  'userFlaggedPrice': '10.00',
  'userFlaggedQuantity': 1}]

In [97]:
# Before conversion
print(f'Total Records: {len(receiptsdf["rewardsReceiptItemList"])}')
print(f'Null Records: {receiptsdf["rewardsReceiptItemList"].isnull().sum()}')

Total Records: 1119
Null Records: 440


In [98]:
receiptsdf['rewardsReceiptItemList']= receiptsdf['rewardsReceiptItemList'].replace(np.nan,0)

In [99]:
# After conversion
print(f'Total Records: {receiptsdf["rewardsReceiptItemList"].count()}')
print(f'Null Records: {receiptsdf["rewardsReceiptItemList"].isnull().sum()}')

Total Records: 1119
Null Records: 0


### Creating new dataframe Ordersdf which has all the products and quantity along with the prices for each item

In [100]:
counter=1
receipt_ids=[]
data=[]
for i in range(len(receiptsdf.rewardsReceiptItemList)):
    if receiptsdf.rewardsReceiptItemList[i] != 0:
        for items in receiptsdf.rewardsReceiptItemList[i]:
            items['receipt_id']=str(counter)
            data.append(items)
        receipt_ids.append(items['receipt_id'])
        counter+=1
    else:
        receipt_ids.append('0')

Ordersdf=pd.DataFrame.from_dict(data)
Ordersdf.head(2)

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.000,4011,True,...,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.000,NaN,NaN,...,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# removing nested JSON from receipts dataframe: "rewardsReceiptItemList"
receiptsdf.drop(columns=['rewardsReceiptItemList'],inplace=True)

# Q.3: Evaluate Data Quality Issues in the Data Provided
identify at least one data quality issue. 

## Step01: Null Check for all the dataframes

### User dataframe ( Primary key: user_id)

In [113]:
print(f'Total records: {len(usersdf)}')
usersdf.isnull().sum()

Total records: 495


active            0
role              0
signUpSource     48
state            56
user_id           0
createdDate       0
lastLoginDate    62
dtype: int64

In Users dataframe, **user_id is the unique and has 0 NULL records** but found missing values in the 2 key columns:
- **state**: region level insights 
- **signupSource**: user acquisition funnel - retention analysis
- **lastLoginDate**: last login date helps to create churn analysis

In [125]:
# Percentage Missing 
print(f"Percentage missing in SignUpsource: {round(usersdf['signUpSource'].isnull().sum()/len(usersdf['signUpSource']) * 100,2)}")
print(f"Percentage missing in State: {round(usersdf['state'].isnull().sum()/len(usersdf['state']) * 100,2)}")


Percentage missing in SignUpsource: 9.7
Percentage missing in State: 11.31


### Brands dataframe

In [127]:
print(f'Total records: {len(brandsdf)}')
brandsdf.isnull().sum()

Total records: 1167


barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brand_id          0
cpg_id            0
cpg_ref           0
brandCode       234
dtype: int64

In Brands dataframe, **brand_id is the unique and has 0 NULL records** but found missing values in the 3 key columns:
- **categoryCode**
- **category**
- **topBrand**

In [140]:
# Percentage Missing 
print(f"Percentage missing in categoryCode: {round(brandsdf['categoryCode'].isnull().sum()/len(brandsdf['categoryCode']) * 100,2)}")
print(f"Percentage missing in category: {round(brandsdf['category'].isnull().sum()/len(brandsdf['category']) * 100,2)}")
print(f"Percentage missing in topBrand: {round(brandsdf['topBrand'].isnull().sum()/len(brandsdf['topBrand']) * 100,2)}")


Percentage missing in categoryCode: 55.7
Percentage missing in category: 13.28
Percentage missing in topBrand: 52.44


Found 52% of top brands boolean data missing

In [151]:
print(f'Total records: {len(brandsdf["topBrand"])}')
print(f'Missing: {brandsdf["topBrand"].isnull().sum()}')
brandsdf.groupby('topBrand')['brand_id'].count()

Total records: 1167
Missing: 612


topBrand
False    524
True      31
Name: brand_id, dtype: int64

### Receipts dataframe

In [152]:
print(f'Total records: {len(receiptsdf)}')
receiptsdf.isnull().sum()

Total records: 1119


bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
receiptId                    0
receipt_createDate           0
receipt_ScannedDate          0
receipt_finishedDate       551
receipt_modifyDate           0
pointsAwardedDate          582
purchaseDate               448
dtype: int64

In receipts dataframe, **receiptId is the primary key(unique) and has 0 NULL records** 

# Converting to csv for further analysis

In [154]:
usersdf.to_csv('users.csv',index=False)
brandsdf.to_csv('brands.csv',index=False)
receiptsdf.to_csv('receipts.csv',index=False)
Ordersdf.to_csv('receipt_orders.csv',index=False)

# Flaws

1. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

-> There is no'Accepted' or 'Rejected' status in the 'rewardsReceiptStatus'. It has below mentioned 5 status with the count of those records
- FINISHED     518
- SUBMITTED    434
- REJECTED      71
- PENDING       50
- FLAGGED       46


In [155]:
receiptsdf.rewardsReceiptStatus.value_counts()

FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: rewardsReceiptStatus, dtype: int64